In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import spline as spl
import newt
from target_func import f as f 

## Ньютон

**Варіант 6**

$$ f(x)=\left\{\begin{array}{l}
(x+1)^{2}+1,-3 \leq x \leq 0 \\
1-8(x-0,5)^{3}, 0 \leq x \leq 1 \\
x-1, \quad 1 \leq x \leq 3
\end{array}\right. $$

**Chebyshev points**
$$ x_{k}=\frac{1}{2}(a+b)+\frac{1}{2}(b-a) \cos \left(\frac{2 k-1}{2 n} \pi\right), \quad k=0, \ldots, n $$

In [2]:
def g(x):
    return (x-3)**2 - 4*np.abs(x-3) + 5

In [3]:
def cheb_nodes(left, right, n):
    return np.array([
        (1/2 * (left + right)) 
            + ((1 / 2 * (right - left)) * math.cos((2 * k - 1) * math.pi / (2 * n)))
           for k in range(1, n + 1)])

def eq_nodes(left, right, n):
    if left > right:
        print("wrong")
    return np.linspace(left, right, n)

In [4]:
n_degree = 10

n_nodes = n_degree + 1
left = -3
right = 3

eq_nodes_x = eq_nodes(left, right, n_nodes)
eq_nodes_y = [f(x) for x in eq_nodes_x]

cheb_nodes_x = cheb_nodes(left, right, n_nodes)
cheb_nodes_y = [f(x) for x in cheb_nodes_x]

In [5]:
# for print
print("Рівновіддалені вузли")
print("{:<8} {:<10} {:<10}".format("i","x_i", "f(x_i)"))
d_eq = dict(zip(eq_nodes_x, eq_nodes_y))
i = 0
for key in d_eq:
    print("{:<8} {:<10} {:<10}".format(i, round(key,2), round(d_eq[key],2)))
    i += 1

Рівновіддалені вузли
i        x_i        f(x_i)    
0        -3.0       5.0       
1        -2.4       2.96      
2        -1.8       1.64      
3        -1.2       1.04      
4        -0.6       1.16      
5        0.0        2.0       
6        0.6        0.99      
7        1.2        0.2       
8        1.8        0.8       
9        2.4        1.4       
10       3.0        2.0       


In [6]:
# for print
print("Чебишовські вузли")
print("{:<8} {:<10} {:<10}".format("i","x_i", "f(x_i)"))
d_eq = dict(zip(cheb_nodes_x[::-1], cheb_nodes_y[::-1]))
i = 0
for key in d_eq:
    print("{:<8} {:<10} {:<10}".format(i, round(key,2), round(d_eq[key],2)))
    i += 1

Чебишовські вузли
i        x_i        f(x_i)    
0        -2.97      4.88      
1        -2.73      3.99      
2        -2.27      2.61      
3        -1.62      1.39      
4        -0.85      1.02      
5        0.0        2.0       
6        0.85       0.67      
7        1.62       0.62      
8        2.27       1.27      
9        2.73       1.73      
10       2.97       1.97      


In [7]:
c_equal = newt.div_diff(eq_nodes_x, eq_nodes_y)[0,:]
c_cheb = newt.div_diff(cheb_nodes_x, cheb_nodes_y)[0,:]

In [8]:
x_data = np.arange(left, right, 0.01)
y_data = np.array([f(x) for x in x_data])

y_data_eq_nodes = np.array([newt.newton_poly(c_equal, eq_nodes_x, x) for x in x_data])
y_data_cheb_nodes = np.array([newt.newton_poly(c_cheb, cheb_nodes_x, x) for x in x_data])

In [9]:
plt.plot(x_data, y_data, label=r"$f(x)$")
plt.plot(x_data, y_data_eq_nodes, label=r"$P_n^E(x)$")
plt.plot(x_data, y_data_cheb_nodes, label=r"$P_n^T(x)$")
plt.xlim(-3, 3)
plt.ylim(-10, 10)
#plt.scatter(eq_nodes_x, eq_nodes_y)
plt.title(f"Інтерполяція при n={n_nodes-1}")
# plt.xlabel("x")
# plt.ylabel("y")
plt.legend()

#plt.savefig(f"equal_nodes_f_and_p_{n_nodes}.png", dpi=300)

Using matplotlib backend: Qt5Agg


In [10]:
plt.plot(x_data, y_data, label=r"$f(x)$")
plt.plot(x_data, y_data_cheb_nodes, label=r"$P_n^T(x)$", zorder=1)
#plt.scatter(cheb_nodes_x, cheb_nodes_y, zorder=2)
plt.title(f"Інтерполяція за чебишовськими вузлами при n={n_nodes-1}")
# plt.xlabel("x")
# plt.ylabel("y")
plt.legend()

#plt.savefig(f"cheb_nodes_f_{n_nodes}.png", dpi=300)

In [11]:
plt.plot(x_data, y_data - y_data_eq_nodes, label=r"$f(x) - P_n^E(x)$", color="tab:orange")
plt.plot(x_data, y_data - y_data_cheb_nodes, label=r"$f(x) - P_n^T(x)$", color="tab:green")
# plt.xlabel("x")
# plt.ylabel("y")
plt.title(f"Похибка n={n_nodes-1}")
plt.axhline(0, color='black', ls="--")
plt.legend()

#plt.savefig(f"equal_nodes_error_{n_nodes}.png", dpi=300)

In [12]:
plt.plot(x_data, y_data - y_data_cheb_nodes, label=r"$f(x) - P_n^T(x)$")
# plt.xlabel("x")
# plt.ylabel("y")
plt.title(f"Похибка за чебишовськми вузлами при n={n_nodes}")
plt.axhline(0, color='black', ls="--")
plt.legend()

#plt.savefig(f"cheb_nodes_error_{n_nodes}.png", dpi=300)

# Сплайни

In [13]:
n_intervals = 50
n = n_intervals + 1

x_nodes = np.linspace(-3, 3, n)
y_nodes = [f(x) for x in x_nodes]

A = spl.mat_a(x_nodes)
H = spl.mat_h(x_nodes)
f_ = y_nodes

m_res = np.linalg.solve(A, H @ f_)

In [14]:
plt.subplots(figsize=(6, 2))
plt.plot(x_nodes, [0 for _ in x_nodes])
plt.scatter(x_nodes, [0 for _ in x_nodes])

$$ Am = Hf$$

In [15]:
y_data_spline = np.array([spl.smoothing_spline3(x, x_nodes, y_nodes, m_res) for x in x_data])

plt.plot(x_data, y_data, label=r"$f(x)$")
plt.plot(x_data, y_data_spline, label=r"$s(x)$")
#plt.scatter(x_nodes, y_nodes)
#plt.xlabel("x")
#plt.ylabel("y")
plt.title(f"Сплайн при {n_intervals} проміжках")
plt.legend()

#plt.savefig(f"interpolation_spline_{n_intervals}.png", dpi=300)

In [16]:
plt.plot(x_data, y_data - y_data_spline, label=r"$f(x)-s(x)$")
plt.axhline(0, color='black', ls="--")
#plt.scatter(x_nodes, y_nodes)
#plt.xlabel("x")
#plt.ylabel("y")
plt.title(f"Похибка при {n_intervals} проміжках сплайну")
plt.legend()

#plt.savefig(f"error_spline_{n_intervals}.png", dpi=300)